## Variable Labeling Ratio with blunt tip.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import json
import sys
sys.path.append("../../../../")

import anamic

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile
import tqdm

data_dir = Path('/home/hadim/Drive/Data/Fake Microtubules/anamic/accuracy/')
data_dir.mkdir(exist_ok=True)

experiment_name = 'labeling_ratio_with_blunt_tip'
experiment_dir = data_dir / experiment_name

../../../../anamic/transformations.py:1916: UserWarning: No module named 'anamic._transformations'
  warnings.warn(str(err))


In [2]:
# Get variable parameters from directory names
dirpaths = list(filter(lambda d: d.is_dir(), experiment_dir.iterdir()))
variable_parameters = [float(d.name.split('-')[1]) for d in dirpaths]
variable_parameter_name = dirpaths[0].name.split('-')[0]

data = []
for variable_parameter, dirpath in tqdm.tqdm_notebook(zip(variable_parameters, dirpaths), total=len(dirpaths)):

    fnames = list(dirpath.glob('*.tif'))
    for image_path in tqdm.tqdm_notebook(fnames, total=len(fnames), leave=False):
        
        # Open the image
        image = tifffile.imread(str(image_path))

        # Load metadata
        metadata = json.load(open(image_path.with_suffix('.json')))

        # Get microtubule tip coordinates
        tip_start = np.array(metadata['tip_start'])
        tip_end = np.array(metadata['tip_end'])
        
        args = {}

        args['get_thick_line_args'] = {}
        args['get_thick_line_args']['length_spacing'] = 5  # pixel
        args['get_thick_line_args']['line_thickness'] = 3000 / metadata['pixel_size']  # pixel

        args['perpendicular_line_fit_args'] = {}
        args['perpendicular_line_fit_args']['length_spacing'] = 0.1  # pixel
        args['perpendicular_line_fit_args']['fit_threshold'] = 0.15
        args['perpendicular_line_fit_args']['continuous_discard'] = False

        args['offset_start'] = 4000 / metadata['pixel_size']  # pixel
        args['offset_end'] = 2000 / metadata['pixel_size']  # pixel

        args['tip_fit_args'] = {}
        args['tip_fit_args']['length_spacing'] = 0.1  # pixel
        args['tip_fit_args']['line_thickness'] = 400 / metadata["pixel_size"]  # pixel
        args['tip_fit_args']['width_spacing'] = 0.1  # pixel
        
        results = anamic.fitter.microtubule_tip_fitter(tip_start, tip_end, image, **args)
        line_fit_tips, x_profile, y_profile, fit_result, fit_func = results
        fit_values = fit_result.values